Scripit to preprocess Hi-C/HiChIP data

In [ ]:
import pandas as pd
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix

- 1. Generate normalized .hic file
- 2. Extract hic matrices:
```
for i in {1..21}
do
    java -jar juicer_tools_1.22.01.jar dump <normalization> NONE imr90.hic $i $i BP 10000 ./zvalue/imr90_chr"$i".txt
done
```
- 3. Run this notebook

### Process from individual chrom text files

In [ ]:

def load_hic(path,hic_resolution):
    hic_dict = {}
    for file in os.listdir(path):
        if file.endswith('txt'):
            print(file)
            chrom = file.split('_')[0]
            print(chrom)
            hic = pd.read_table(path+file, header = None)
            start = hic[[0,1]].values.min()
            print(start)
            end = hic[[0,1]].values.max()+hic_resolution
            hic = hic.pivot(0,1,2).fillna(0)
            hic = hic.reindex(columns=np.arange(0,end, hic_resolution).astype(int), index=np.arange(0,end, hic_resolution).astype(int),
                      fill_value = 0)
            hic_dict[chrom] = csr_matrix(hic)

    return hic_dict

In [ ]:
hic_resolution = 1e4

path = './zvalue/'
hic_dict = load_hic(path,hic_resolution=hic_resolution)
pickle.dump(hic_dict, open("./zvalue/hic_zscore_dict.p", "wb"), protocol=4)  # save it into a file named save.p


In [ ]:
hic_resolution = 1e4

path = './qvalue/'
hic_dict = load_hic(path,hic_resolution=hic_resolution)
pickle.dump(hic_dict, open("./qvalue/hic_qvalue_dict.p", "wb"), protocol=4)  # save it into a file named save.p
